In [1]:
import sqlite3
import pandas as pd
import numpy as np  
import math
import random

In [2]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()
conn.create_function("SQRT", 1, math.sqrt)
conn.create_function("FLOOR", 1, math.floor)
def rand_func(dummy):
    return random.randint(1, 50)
conn.create_function("RAND",1,rand_func)

In [3]:
#Tạo bảng
cursor.execute("""
CREATE TABLE customer_purchases (
    market_date DATE,
    vendor_id INT,
    quantity INT,
    cost_to_customer_per_qty DECIMAL(10, 2)
);
""")
cursor.execute("""
INSERT INTO customer_purchases (market_date, vendor_id, quantity, cost_to_customer_per_qty)
VALUES
    ('2019-03-02', 8, 2, 4.00),
    ('2019-03-02', 8, 1, 4.00),
    ('2019-03-09', 8, 1, 4.00),
    ('2019-03-09', 9, 1, 16.00),
    ('2019-03-09', 9, 1, 18.00),
    ('2019-03-02', 4, 5, 2.00),
    ('2019-03-02', 4, 8, 2.00),
    ('2019-03-02', 4, 1, 2.00),
    ('2019-03-09', 4, 10, 2.00),
    ('2019-03-02', 1, 1, 5.50),
    ('2020-03-01', 8, 3, 4.50),
    ('2020-03-08', 9, 2, 5.00),
    ('2020-03-01', 4, 5, 2.50),
    ('2020-03-08', 4, 7, 2.20),
    ('2021-03-06', 8, 2, 5.00),
    ('2021-03-13', 9, 1, 17.00),
    ('2021-03-06', 4, 8, 2.00),
    ('2021-03-13', 4, 7, 2.50),
    ('2022-03-05', 8, 3, 4.20),
    ('2022-03-12', 9, 2, 5.50),
    ('2022-03-05', 4, 5, 2.10),
    ('2022-03-12', 4, 6, 2.30);

""")
conn.commit()

In [4]:
cursor.execute("""
CREATE TABLE market_date_info (
    market_date DATE,
    market_year INT,
    market_week INT
);
""")
# cursor.execute("""
# INSERT INTO market_date_info (market_date, market_year, market_week)
# SELECT 
#     market_date,
#     EXTRACT(YEAR FROM market_date) AS market_year,
#     EXTRACT(WEEK FROM market_date) AS market_week
# FROM customer_purchases
# """)
# conn.commit()

In [5]:
cursor.execute("""
INSERT INTO market_date_info (market_date, market_year, market_week)
SELECT 
    market_date,
    CAST(strftime('%Y', market_date) AS INTEGER) AS market_year,
    CAST(strftime('%W', market_date) AS INTEGER) AS market_week
FROM customer_purchases
""")
conn.commit()

In [6]:
df = 'select * from market_date_info order by market_date asc'
df = pd.read_sql_query(df, conn)
df

,market_date,market_year,market_week
0,2019-03-02,2019,8
1,2019-03-02,2019,8
2,2019-03-02,2019,8
3,2019-03-02,2019,8
4,2019-03-02,2019,8
5,2019-03-02,2019,8
6,2019-03-09,2019,9
7,2019-03-09,2019,9
8,2019-03-09,2019,9
9,2019-03-09,2019,9


In [7]:
df = '''
SELECT MIN(cp.market_date) AS first_market_date_of_week,
       ROUND(SUM(cp.quantity * cp.cost_to_customer_per_qty), 2) AS weekly_sales
FROM customer_purchases AS cp
JOIN market_date_info AS md
ON cp.market_date = md.market_date
GROUP BY md.market_year, md.market_week
ORDER BY md.market_year, md.market_week
'''
df = pd.read_sql_query(df, conn)
df

,first_market_date_of_week,weekly_sales
0,2019-03-02,273.0
1,2019-03-09,232.0
2,2020-03-01,52.0
3,2020-03-08,50.8
4,2021-03-06,52.0
5,2021-03-13,69.0
6,2022-03-05,46.2
7,2022-03-12,49.6


In [8]:
cursor.execute('ALTER TABLE customer_purchases ADD COLUMN product_id INT;')
cursor.execute('ALTER TABLE customer_purchases ADD COLUMN customer_id INT;')

In [9]:
# cursor.execute('''UPDATE customer_purchases
#                   SET 
#                       customer_id = FLOOR(RANDOM()) + 1,
#                       product_id = FLOOR(RANDOM()) + 1;
#                ''')
cursor.execute('''UPDATE customer_purchases
                  SET 
                      customer_id = RAND(0),
                      product_id = RAND(0);
               ''')

In [10]:
df = 'select * from customer_purchases'
df = pd.read_sql_query(df, conn)
df

,market_date,vendor_id,quantity,cost_to_customer_per_qty,product_id,customer_id
0,2019-03-02,8,2,4.0,12,48
1,2019-03-02,8,1,4.0,19,13
2,2019-03-09,8,1,4.0,39,35
3,2019-03-09,9,1,16.0,34,12
4,2019-03-09,9,1,18.0,18,1
5,2019-03-02,4,5,2.0,49,50
6,2019-03-02,4,8,2.0,37,38
7,2019-03-02,4,1,2.0,26,26
8,2019-03-09,4,10,2.0,1,2
9,2019-03-02,1,1,5.5,48,32


In [11]:
# cursor.execute('''
# -- B1: Tạo bảng tạm chứa danh sách các lần mua hàng duy nhất của từng khách hàng
# WITH customer_markets_attended AS (
#     SELECT DISTINCT 
#         customer_id, 
#         market_date
#     FROM 
#         customer_purchases
#     ORDER BY 
#         customer_id, 
#         market_date
# )
# ''')

In [12]:
cursor.execute('''
CREATE TABLE customer_markets_attended AS
SELECT DISTINCT 
    customer_id, 
    market_date
FROM 
    customer_purchases
ORDER BY 
    customer_id, 
    market_date
''')


In [13]:
df = '''
SELECT  market_date, 
        customer_id,
        quantity * cost_to_customer_per_qty AS purchase_total,
        COUNT(DISTINCT vendor_id) AS vendor_count,
        COUNT(DISTINCT customer_id) AS customer_count
FROM    customer_purchases
Group by market_date, customer_id
order by market_date, customer_id

'''
df = pd.read_sql_query(df, conn)
df


,market_date,customer_id,purchase_total,vendor_count,customer_count
0,2019-03-02,13,4.0,1,1
1,2019-03-02,26,2.0,1,1
2,2019-03-02,32,5.5,1,1
3,2019-03-02,38,16.0,1,1
4,2019-03-02,48,8.0,1,1
5,2019-03-02,50,10.0,1,1
6,2019-03-09,1,18.0,1,1
7,2019-03-09,2,20.0,1,1
8,2019-03-09,12,16.0,1,1
9,2019-03-09,35,4.0,1,1


In [14]:
# df = '''
# -- Tìm ngày mua tiếp theo của cùng 1 khách hàng sau ngày hiện tại(market_date).
# Select 
# 	T1.customer_id,
#     T1.market_date as ngay_hien_tai,
#     Min(T2.market_date) as ngay_tiep_theo,
#     DATEDIFF('DAT', ngay_hien_tai, ngay_tiep_theo) as so_ngay
# from customer_markets_attended AS T1,
# 	 customer_markets_attended AS T2
# where T1.market_date < T2.market_date and
#       T1.customer_id = T2.customer_id
# GROUP BY 
#     T1.customer_id, T1.market_date;

# '''
# df = pd.read_sql_query(df, conn)
# df

In [15]:
df = '''
-- Tìm ngày mua tiếp theo của cùng 1 khách hàng sau ngày hiện tại (market_date)
SELECT 
    T1.customer_id,
    T1.market_date AS ngay_hien_tai,
    MIN(T2.market_date) AS ngay_tiep_theo,
    JULIANDAY(MIN(T2.market_date)) - JULIANDAY(T1.market_date) AS so_ngay
FROM 
    customer_markets_attended AS T1,
    customer_markets_attended AS T2
WHERE 
    T1.customer_id = T2.customer_id AND
    T1.market_date < T2.market_date
GROUP BY 
    T1.customer_id, T1.market_date;
'''
df = pd.read_sql_query(df, conn)
df


,customer_id,ngay_hien_tai,ngay_tiep_theo,so_ngay
0,13,2019-03-02,2022-03-12,1106.0
1,15,2020-03-08,2021-03-13,370.0
2,26,2019-03-02,2020-03-08,372.0
3,26,2020-03-08,2022-03-12,734.0
4,32,2019-03-02,2021-03-06,735.0


In [17]:
df = '''
WITH next_purchases AS (
    SELECT 
        T1.customer_id,
        T1.market_date AS ngay_hien_tai,
        MIN(T2.market_date) AS ngay_tiep_theo,
        JULIANDAY(MIN(T2.market_date)) - JULIANDAY(T1.market_date) AS so_ngay
    FROM 
        customer_markets_attended AS T1,
        customer_markets_attended AS T2
    WHERE 
        T1.customer_id = T2.customer_id AND
        T1.market_date < T2.market_date
    GROUP BY 
        T1.customer_id, T1.market_date
)
SELECT 
    customer_id,
    ngay_hien_tai,
    ngay_tiep_theo,
    CAST(so_ngay <= 30 AS INTEGER) AS quay_lai_trong_30_ngay
FROM 
    next_purchases;

'''
df = pd.read_sql_query(df, conn)
df


,customer_id,ngay_hien_tai,ngay_tiep_theo,quay_lai_trong_30_ngay
0,13,2019-03-02,2022-03-12,0
1,15,2020-03-08,2021-03-13,0
2,26,2019-03-02,2020-03-08,0
3,26,2020-03-08,2022-03-12,0
4,32,2019-03-02,2021-03-06,0
